
GTFS Wiki: http://wiki.corp.nextbus.com/nextbuswiki/GtfsChecks (need to be on NexBus VPN) 

GTFS Goolge Guideline: https://developers.google.com/transit/gtfs/reference/

# <span style="color:red">Import the needed packages</span>

In [1]:
import pandas as pd
import numpy as np
#import pixiedust as pix # will look into the capabilites of this library

### <span style="color:green">Add the location of your GTFS data</span>

In [2]:
# for windows
# filepath = r"\\cts-con-fld-001\users$\207329\My Documents\qa\vacaville\vacavillecitycoach-ca-us_20181015"

# for linux
filepath = r"/mnt/shared/qa/tahoe"

# filepath2 = r"/mnt/shared/qa/seattle-sc/Seattle-sc-ALL_20181126_google_daily_transit"



'''Some functions to make life eaiser
Note if you are using Windows, you'll need to modify the "/" '''




# load file and show top 5 records
def load(fName): 
    f = filepath + "/" + "{}.txt".format(fName)
    global df
    df = pd.read_csv(f)
    return df.info()

# load second file for comparision
def load2(fName): 
    f = filepath2 + "/" + "{}.txt".format(fName)
    global df2
    df2 = pd.read_csv(f)
    return df2.info()

import csv
# short save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '//' + "{}".format(fName), sep =',', index=False)#, quoting=csv.QUOTE_NONNUMERIC)
    return print("Saved {} to {}".format(fName,filepath))

# short find null
def null(fName):
    return df[(fName).isnull()]

# drop field (field name, file name)
def drop(fName='timepoint',fileName='stop_times'):
    df.drop(fName, 1, inplace=True)
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Dropped '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(fName,fileName))

# drop dups in blocks.txt for CNB 
def dups():
    #df[df.block_id.duplicated()]
    df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
    return df1.info()
    
# sort sequence for stop_times.txt file
def sort():
    df.sort_values(['trip_id', 'stop_sequence'])
    return df.head(15)

# shape less than certain amount of points for shapes
def lessthan(x):
    df1 = df.groupby(['shape_id']).count()
    return df1[(df1.shape_pt_lat < x)]

# function to move copy stop_code to stop_id in the stops.txt file
def copyField(newField='stop_code',origField='stop_id',fileName='stops'):
    df[newField] = df[origField]
    df.to_csv(filepath + '//' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Copied the '{}' field to the '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(origField,newField,fileName))




### work for CNB shape issues
# def shapeFix(indx1,indx2,shapeName):
#     x1 = int(indx1)
#     x2 = int(indx2)
#     global df
#     df = df.drop(df.index[x1:x2]) # drop double shape pnts 
#     df = df[(df.shape_id == '{}'.format(shapeName))] # create new df of just shape pnts
#     df['shape_pt_sequence'] = range(1,1 + len(df)) # re-sequence shape pnts
#     df_cut = df # give unique name to subset of shape pnts

#     # load orig data, drop all of courthou1, append 
#     load('shapes') # load orig df
#     df = df[df.shape_id != '{}'.format(shapeName)] # drop all cases of shape 
#     global df_all
#     df_all = df.append(df_cut,ignore_index=True) # create new df and append df_cut
#     return  print('use df_all')



## Random Work-flows

In [ ]:
## fill na with number, query and output file  
df['block_id'].fillna(999999, inplace=True)
df1 = df[(df.block_id == 999999)]
df['block_id'].unique()
save(df1,'trips_missingBlocks.txt')

# compare two files
# set the field to a list
oldTrips = df2['trip_id'].unique().tolist()
newTrips = df['trip_id'].unique().tolist()

oldBlocks = df2['block_id'].unique().tolist()
newBlocks = df['block_id'].unique().tolist()

# use set and difference to compare the two list
set(oldTrips).difference(newTrips)

set(newBlocks).difference(oldBlocks)

# can alsow use this logic... but the above works just fine
if (40803723 in newTrips):
    print ("Exist")
else:
    print("Nope")

##### seattle-sc 'like' function 
df=df.loc[df['route_id'].isin([100340,102638])]
df
save(df,'trips')
df['service_id'].unique()
df2=df2.loc[df2['route_id'].isin([100340,102638])]

#### create new column based on conditions of other columns
df['timeCheck'] = np.where(df['arrival_time']==df['departure_time'],'yes','no')
df1 = df[(df.timeCheck=='no')]
df['timeCheck'].unique()

# syntax for leading zeros (4 for example)
df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))


# <span style="color:red">agency.txt</span>

In [3]:
load('agency')
df.head(25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
agency_id          1 non-null int64
agency_url         1 non-null object
agency_lang        1 non-null object
agency_name        1 non-null object
agency_phone       1 non-null object
agency_timezone    1 non-null object
agency_fare_url    0 non-null float64
dtypes: float64(1), int64(1), object(5)
memory usage: 136.0+ bytes


,agency_id,agency_url,agency_lang,agency_name,agency_phone,agency_timezone,agency_fare_url
0,904,http://www.placer.ca.gov/Departments/Works/Tra...,en,Tahoe Truckee Area Regional Transit,(530) 550-1212,America/Los_Angeles,NaN


# <span style="color:red">routes.txt</span>

### Required Fields: 
1. route_id- These are the routes that are shown to the public. Route ID's cannot have leading zeros, asterisks, underscore, dashes and perhaps other characters. 
2. route_type- numberic value with specific meanings:
    * 0 Tram, Streetcar, Light rail
    * 1 Subway, Metro
    * 2 Rail
    * 3 Bus
    * 4 Ferry
    * 5 Cable Car

In [4]:
# just use the funtion 'load()' to load the .txt file and see the first 5 lines ex: loadfile('routes')
load('routes')
df.head(20)
#copyField('route_short_name','route_id','routes')
# dfEOS = df[(df.agency_id == 'EOS')]
# save(dfEOS,'routes')

#### seattle-sc 'like' function
#df = df.loc[df['agency_id']=='EOS']
# df1['route_long_name'] = df1['route_short_name']
# df = df1
# save('routes_NEW.txt')
#df[(df.route_id==100249)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 12 columns):
agency_id                 10 non-null int64
route_id                  10 non-null int64
route_short_name          9 non-null object
route_long_name           2 non-null object
route_desc                0 non-null float64
route_type                10 non-null int64
route_url                 1 non-null object
route_color               10 non-null object
route_text_color          10 non-null object
route_sort_order          10 non-null int64
min_headway_minutes       0 non-null float64
eligibility_restricted    10 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 1.0+ KB


,agency_id,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,route_sort_order,min_headway_minutes,eligibility_restricted
0,134,742,225,NaN,NaN,3,NaN,339999,ffffff,0,NaN,0
1,134,840,226,NaN,NaN,3,NaN,339999,ffffff,1,NaN,0
2,134,744,Silver,NaN,NaN,3,NaN,808080,000000,2,NaN,0
3,134,745,White,NaN,NaN,3,NaN,000000,ffffff,3,NaN,0
4,134,743,Blue,NaN,NaN,3,NaN,000099,ffffff,4,NaN,0
5,134,747,Green,Eastview,NaN,3,NaN,00cc33,ffffff,5,NaN,0
6,134,746,Green,NaN,NaN,3,NaN,00854a,ffffff,6,NaN,0
7,134,749,Orange,NaN,NaN,3,NaN,ff3300,ffffff,7,NaN,0
8,134,748,Gold,NaN,NaN,3,NaN,ffff00,000000,8,NaN,0
9,134,1112,NaN,Ridgecrest Shuttle,NaN,3,http://www.palosverdes.com/pvtransit/SCHEDULES...,3f669f,ffffff,9,NaN,0


# <span style="color:red">trips.txt</span>

### Required Fields:
1. route_id- Check to see if there are trips for every route. 
2. service_id- Check to see if the value of this field comes from the calendar.txt file. (below)
3. trip_id
4. direction_id- Can either be 1 or 0. Can omit if all routes are in one direction. 
5. block_id- GTFS list this as optional, NextBus requires it. Make sure there are no missig values. 
6. shape_id- GTFS list this as optional, NextBus requires it. This field links with the shapes.txt file.(below)
   


In [4]:
load('trips')
df.head()
# df.block_id.unique().tolist()
# dffoo = df[(df.block_id == 5254117)]
# dffoo.head()

## dups work
#df[df.block_id.duplicated()]
#df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
#102638

# df1 = df[(df.route_id == '860-150') | (df.route_id == '861-150')]
# df1 = df[(df.trip_headsign == 'Not In Service')]
# df1
# save(df1,'foothill_trips_notInService.txt')




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 18 columns):
route_id                       321 non-null object
service_id                     321 non-null object
trip_id                        321 non-null object
trip_short_name                0 non-null float64
trip_headsign                  310 non-null object
direction_id                   310 non-null float64
block_id                       321 non-null int64
shape_id                       321 non-null object
bikes_allowed                  0 non-null float64
wheelchair_accessible          0 non-null float64
trip_type                      0 non-null float64
drt_max_travel_time            130 non-null object
drt_avg_travel_time            130 non-null object
drt_advance_book_min           130 non-null float64
drt_pickup_message             0 non-null float64
drt_drop_off_message           0 non-null float64
continuous_pickup_message      0 non-null float64
continuous_drop_off_message    0 n

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type,drt_max_travel_time,drt_avg_travel_time,drt_advance_book_min,drt_pickup_message,drt_drop_off_message,continuous_pickup_message,continuous_drop_off_message
0,TLR,c_15569_b_19557_d_127,t_375279_b_19557_tn_0,NaN,Westbound,0.0,10,p_375044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TLR,c_15569_b_19557_d_127,t_375277_b_19557_tn_0,NaN,Westbound,0.0,10,p_375044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TLR,c_15569_b_19557_d_127,t_375276_b_19557_tn_0,NaN,Westbound,0.0,10,p_375044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TLR,c_15569_b_19557_d_127,t_375275_b_19557_tn_0,NaN,Westbound,0.0,10,p_375044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TLR,c_15569_b_19557_d_127,t_375286_b_19557_tn_0,NaN,Westbound,0.0,10,p_375047,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# <span style="color:red">stops.txt</span>

### Required Fields:
1. stop_id 
2. stop_lat- Make sure none are zero, or empty.
3. stop_lon- Make sure none are zero, or empty.
    - make sure lat and long are at least five decimal places (most do six) 

In [9]:
load('stops')
# df.head(15)
# copyField('stop_code','stop_id','stops')
copyField()

# copyField('stop_code','stop_id','stops')

#### seattle-sc 'like' function
#df_stop = df.loc[df['stop_code'].isin(stopList)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 15 columns):
stop_id                308 non-null int64
stop_code              0 non-null float64
platform_code          0 non-null float64
stop_name              308 non-null object
stop_desc              0 non-null float64
stop_lat               308 non-null float64
stop_lon               308 non-null float64
zone_id                0 non-null float64
stop_url               0 non-null float64
location_type          308 non-null int64
parent_station         0 non-null float64
stop_timezone          308 non-null object
position               0 non-null float64
direction              0 non-null float64
wheelchair_boarding    308 non-null int64
dtypes: float64(10), int64(3), object(2)
memory usage: 36.2+ KB
Copied the 'stop_id' field to the 'stop_code' field.
Saved 'stops_FIXED.txt' to file folder.


# <span style="color:red">stop_times.txt</span>

### Required Fields:
1. trip_id
2. arrival_time- Should be blank for non-scheduled stops
3. depature_times- Should be blank for non-scheduled stops
4. stop_id
5. stop_sequence- Some agencies will sort this field incorrectly, putting all the "1s" first.
    - note* watch out for the timepoints field, if there is a 1, then it is an acutal stop. If there are not enough of 1s, this may cause problems. It's okay to delete this field. 

In [6]:
load('stop_times_FIXED') 
# df.head()
# drop()
# df.head(100)
# syntax for leading zeros (4 for example)
# df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))
# df.head(100)
# df['timepoint'].unique()

# df[(df.arrival_time!=null) & (df.timepoint==1)]
# df[(df.trip_id == 't_31175_b_718_tn_0')]





#### seattle-sc 'like' function
# df_trip = df.loc[df['trip_id'].isin(tripsList)]
# df_trip.sort_values(['trip_id', 'stop_sequence'])
# stopList = df_trip['stop_id'].unique().tolist()
# save(df_trip,'stop_times_NEW.txt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 19 columns):
trip_id                         8884 non-null object
arrival_time                    3578 non-null object
departure_time                  3578 non-null object
stop_id                         8884 non-null int64
stop_sequence                   8884 non-null int64
stop_headsign                   0 non-null float64
pickup_type                     8884 non-null int64
drop_off_type                   8884 non-null int64
shape_dist_traveled             8884 non-null float64
start_service_area_id           0 non-null float64
end_service_area_id             0 non-null float64
start_service_area_radius       0 non-null float64
end_service_area_radius         0 non-null float64
continuous_pickup               8884 non-null int64
continuous_drop_off             8884 non-null int64
pickup_area_id                  0 non-null float64
drop_off_area_id                0 non-null float64
pickup_serv

# <span style="color:red">shapes.txt</span>

### Required Fields:
1. shape_id
2. shape_pnt_lat- at least five decimal places
3. shape_pnt_lon- at least five deciaml places

In [12]:
load('shapes')
# df.head(50)
df.describe()

# lessthan(100)


# df['shape_pt_sequence'] = df['shape_pt_sequence'].astype(np.int64)
# save(df,'shapesINT.txt')
# df1 = df[(df.shape_id=='J_trip_inbound_pm')]
# save(df1,'J_trip_inbound_pm.csv')

# flip the point sequence 
# df2 = df[(df.shape_id == 'J_trip_inbound_am')]
# drop = df2[::-1].drop(columns=['shape_pt_sequence'])
# drop.loc[:,'shape_pt_sequence'] = range(1, 1 + len(drop))
# # drop

# df3 = df[df.shape_id != 'J_trip_inbound_am']
# df4 = df3.append(drop)
# new = df4[(df4.shape_id == 'J_trip_inbound_am')]
# df4
# save(df4,'shapes_NEW.txt')
# dfJ = df[(df['shape_id'].str.contains('J'))]
# dfJ['shape_id'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76054 entries, 0 to 76053
Data columns (total 5 columns):
shape_id               76054 non-null int64
shape_pt_lat           76054 non-null float64
shape_pt_lon           76054 non-null float64
shape_pt_sequence      76054 non-null int64
shape_dist_traveled    76054 non-null float64
dtypes: float64(3), int64(2)
memory usage: 2.9 MB


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
count,7.605400e+04,76054.000000,76054.000000,76054.000000,76054.000000
mean,4.274056e+06,34.057977,-117.927061,171428.483551,10.501442
std,2.027416e+06,0.040381,0.130085,146777.324267,8.455084
min,1.780108e+06,33.914790,-118.268027,10001.000000,0.000000
25%,2.700074e+06,34.030039,-118.001340,50057.000000,4.019000
50%,4.820110e+06,34.062579,-117.929118,120589.000000,8.306500
75%,4.980118e+06,34.079085,-117.829664,250015.000000,14.885000
max,8.610010e+06,34.151759,-117.689463,710012.000000,41.585000


# <span style="color:red">calendar.txt</span>

### Required:
Run the calendar script if more than one service record is defined for the day of the week.
* If there is more than one service defined for the week, then run the calendar.py script.


In [18]:
load('calendar')
df.head(35)


#### seattle-sc 'like' function
# df_cal = df.loc[df['service_id'].isin(serviceList)]
# df_cal

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 11 columns):
service_id      3 non-null object
service_name    3 non-null object
monday          3 non-null int64
tuesday         3 non-null int64
wednesday       3 non-null int64
thursday        3 non-null int64
friday          3 non-null int64
saturday        3 non-null int64
sunday          3 non-null int64
start_date      3 non-null int64
end_date        3 non-null int64
dtypes: int64(9), object(2)
memory usage: 344.0+ bytes


,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,c_313_b_718_d_31,Year Round (Weekday),1,1,1,1,1,0,0,20150102,20200101
1,c_2173_b_21609_d_31,Early Summer (Weekday),1,1,1,1,1,0,0,20190617,20190712
2,c_2158_b_21608_d_31,Summer Service (Weekday),1,1,1,1,1,0,0,20190617,20190725


# <span style="color:red">calendar_dates.txt</span>

### Required:
This is an optional file, but is used to mark special holidays. Make sure the dates make sense.

In [16]:
load('calendar_dates')
df.head(50)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
service_id        12 non-null object
date              12 non-null int64
exception_type    12 non-null int64
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


,service_id,date,exception_type
0,1901-FT1905-Sunday-82,20190704,1
1,1901-FT1905-Sunday-82,20190902,1
2,1901-FT1905-Weekday-82,20190704,2
3,1901-FT1905-Weekday-82,20190902,2
4,1901-Pomona-Sunday-62,20190704,1
5,1901-Pomona-Sunday-62,20190902,1
6,1901-Pomona-Weekday-62,20190704,2
7,1901-Pomona-Weekday-62-1111000,20190704,2
8,1901-Pomona-Weekday-62-0000100,20190704,2
9,1901-Pomona-Weekday-62,20190902,2
